# Create training data for merchant categories
For this, I have used google places api, to label merchant categories
In order to do this, make sure google maps api client library is installed:

    pip install -U googlemaps

## All python imports and globals

In [81]:
%matplotlib inline
import bank_csv_parser as bp
import pandas as pd
import googlemaps
import api_keys as apiKeys
import json
import difflib

DATASET_FILE_CBA ='../data/CBA.csv'
DATASET_FILE_BENDIGO ='../data/Bendigo.csv'
DATASET_FILE_ANZ ='../data/ANZ.csv'

DATASET_OUTPUT_FILE_MERCHANT_CATEGORIES='../data/MerchantCategories.csv'

COL_NAME_MERCHANT='Merchant'
COL_NAME_SUBURB='Suburb'

COL_NAME_GOOGLE_PLACE_NAME='GPlace_Name'
COL_NAME_GOOGLE_PLACE_ID='GPlace_Id'
COL_NAME_GOOGLE_ADDR='GAddr'
COL_NAME_GOOGLE_CAT0='GCat0'
COL_NAME_GOOGLE_CAT1='GCat1'
COL_NAME_GOOGLE_CAT2='GCat2'


#initialize google maps api client
gmaps = googlemaps.Client(key=apiKeys.API_KEY_GOOGLE_PLACES)


## Prepare merchant list by combining normalized data from all banks
Normalize all bank datasets and retain columns with distinct values for:

    Merchant | Suburb
    
Then, use google places api to extend this to:

    Merchant | Suburb | GPlace_Name | GPlace_Id| GAddr | GCat0 | GCat1 | GCat2
    

Google categories are defined in: https://developers.google.com/places/web-service/supported_types. Some of them wouldn't make sense for our income/expense categorization app. So, only Table1 categories are whitelisted, and from them the following are excluded:

    * establishment


In [4]:
CATEGORY_WHITELIST = [
    'accounting',
    'airport',
    'amusement_park',
    'aquarium',
    'art_gallery',
    'atm',
    'bakery',
    'bank',
    'bar',
    'beauty_salon',
    'bicycle_store',
    'book_store',
    'bowling_alley',
    'bus_station',
    'cafe',
    'campground',
    'car_dealer',
    'car_rental',
    'car_repair',
    'car_wash',
    'casino',
    'cemetery',
    'church',
    'city_hall',
    'clothing_store',
    'convenience_store',
    'courthouse',
    'dentist',
    'department_store',
    'doctor',
    'electrician',
    'electronics_store',
    'embassy',
    'finance',
    'fire_station',
    'florist',
    'food',
    'funeral_home',
    'furniture_store',
    'gas_station',
    'general_contractor',
    'grocery_or_supermarket',
    'gym',
    'hair_care',
    'hardware_store',
    'health',
    'hindu_temple',
    'home_goods_store',
    'hospital',
    'insurance_agency',
    'jewelry_store',
    'laundry',
    'lawyer',
    'library',
    'liquor_store',
    'local_government_office',
    'locksmith',
    'lodging',
    'meal_delivery',
    'meal_takeaway',
    'mosque',
    'movie_rental',
    'movie_theater',
    'moving_company',
    'museum',
    'night_club',
    'painter',
    'park',
    'parking',
    'pet_store',
    'pharmacy',
    'physiotherapist',
    'place_of_worship',
    'plumber',
    'police',
    'post_office',
    'real_estate_agency',
    'restaurant',
    'roofing_contractor',
    'rv_park',
    'school',
    'shoe_store',
    'shopping_mall',
    'spa',
    'stadium',
    'storage',
    'store',
    'subway_station',
    'synagogue',
    'taxi_stand',
    'train_station',
    'transit_station',
    'travel_agency',
    'university',
    'veterinary_care',
    'zoo'    
]

## Combine all merchant and suburb data

In [16]:
COLS_TO_RETAIN=[COL_NAME_MERCHANT, COL_NAME_SUBURB]
cbaDf = bp.loadAndNormalize(DATASET_FILE_CBA).ix[:,COLS_TO_RETAIN]
bendigoDf = bp.loadAndNormalize(DATASET_FILE_BENDIGO).ix[:,COLS_TO_RETAIN]
anzDf = bp.loadAndNormalize(DATASET_FILE_ANZ).ix[:,COLS_TO_RETAIN]

df = pd.concat([cbaDf, bendigoDf, anzDf])
df.drop_duplicates(subset=COLS_TO_RETAIN, inplace=True)
print df.shape
df.head()

/home/juicebox/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  from ipykernel import kernelapp as app
/home/juicebox/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  app.launch_new_instance()


(194, 2)


/home/juicebox/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


,Merchant,Suburb
0,HARVIN SPICES,HURSTVILLE BC
1,DELISSE,SYDNEY
2,SOUTHGATE GROUP AUSTRA,SYDNEY
3,Deli Ziosa,SYDNEY
5,BING LEE HURSTVILLE,HURSTVILLE BC


## Attach Google Places API results 
Call places api, extract resutl and add new columns. When no results found from google, set value to `'UNCATEGORIZED'`. 
Sometimes Google places api returns multiple matches. In such cases, a selection will be made based on the closest match between field `'name'` (from google json) and `Merchant` text.

The google search will use the string combination of `'Merchant'` + `', '` + `'Suburb'`. So, of Merchant is `'DELISSE'` and Suburb is `'SYDNEY'`, the api query will be for `'DELISSE, SYDNEY'` to help Google narrow down the place.

When there is no match for a lengthier string, say Merchant is `'LITTLE INDIA - HARRIS'`, and Suburb is `'HARRIS PARK'`, the search will trim off to the second lengthiest segment of the merchant. In other words:

* First attempt: No results for `LITTLE INDIA - HARRIS, HARRIS PARK`
    * Second attempt: `LITTLE INDIA, HARRIS PARK`
        * Third attempt (if second was unsuccessful): `LITTLE, HARRIS PARK`

In [191]:
LAT_LNG_APPROX_SYDNEY_CENTER=(-33.861720, 151.012569)
def findNearSydney(merchant, suburb):
    resultJson = gmaps.places_nearby(location=LAT_LNG_APPROX_SYDNEY_CENTER,
                       rank_by='distance',
                       keyword='{}, {}'.format(merchant, suburb))
    return resultJson
                                    

def getBestMatch(merchant, jsonResult):
    candidates = {}
    rNames = []
    rName = None
    lastMatch = None
    for r in jsonResult['results']:
        rName = r['name'].encode('utf-8')
        rVicinity = r['vicinity'].encode('utf-8')
        rNameWithVicinity = "{} {}".format(rName, rVicinity)
        rNames.append(rNameWithVicinity)
        types = [t.encode('utf-8') for t in r['types'] if t in CATEGORY_WHITELIST][:3] + [None, None, None]
        types = types[:3]
        candidates[rNameWithVicinity] = (rName, r['place_id'].encode('utf-8'), rVicinity, types)
        lastMatch = candidates[rNameWithVicinity]

    match = lastMatch
    if len(rNames)>1:
        matches = difflib.get_close_matches(merchant, rNames)
        if matches:
            match = candidates[matches[0]]
    if match:
        return match
    else:
        return (rName, None, None, [None, None, None])

    
def getCategories(merchant, suburb):
    uncat = 'UNCATEGORIZED'
    unknown = 'UNKNOWN'
    result = findNearSydney(merchant,suburb)
    if result['status'] == 'ZERO_RESULTS':
        return unknown, unknown, unknown, [uncat, uncat, uncat]    
    bizName, placeId, addr, cats = getBestMatch("{}, {}".format(merchant,suburb), result)
    if not placeId:
        placeId = unknown
    if not addr:
        addr = unknown
    if not cats:
        cats = [uncat, uncat, uncat]
    elif not cats[0]:
        cats[0] = uncat
    elif not cats[1]:
        cats[1] = uncat
    elif not cats[2]:
        cats[2] = uncat
    return bizName, placeId, addr, cats
    
    


In [167]:
litIndia = getCategories('Little India harris', 'harris park')

In [182]:
delisse = getCategories('Delisse', 'sydney')
delisse

Cats  ['cafe', 'bakery', 'restaurant']


('Delisse',
 'ChIJsWvvmWquEmsR_6P02hd1eGQ',
 '52 Martin Place, Sydney',
 ['cafe', 'bakery', 'restaurant'])

In [169]:
litIndia

('Little India',
 'ChIJfZmJoSOjEmsRcOWkBJCreyI',
 'Harris Park',
 ['grocery_or_supermarket', 'food', 'store'])

In [203]:
def addColumnsForPlaces(df):
    def addLastColumn(df, colName, colVal):
        if not colName in df.columns:
            df.insert(len(df.columns), colName, colVal)
    #Create new columns
    addLastColumn(df, COL_NAME_GOOGLE_PLACE_NAME, 'UNKNOWN')
    addLastColumn(df, COL_NAME_GOOGLE_PLACE_ID, 'UNKNOWN')
    addLastColumn(df, COL_NAME_GOOGLE_ADDR, 'UNKNOWN')
    addLastColumn(df, COL_NAME_GOOGLE_CAT0, 'UNCATEGORIZED')
    addLastColumn(df, COL_NAME_GOOGLE_CAT1, 'UNCATEGORIZED')
    addLastColumn(df, COL_NAME_GOOGLE_CAT2, 'UNCATEGORIZED')
    #add values into these columns
    for idx,row in df.iterrows():
        merchant = row[COL_NAME_MERCHANT]
        suburb = row[COL_NAME_SUBURB]
        place = getCategories(merchant, suburb)
        df.at[idx, COL_NAME_GOOGLE_PLACE_NAME] = place[0]
        df.at[idx, COL_NAME_GOOGLE_PLACE_ID] = place[1]
        df.at[idx, COL_NAME_GOOGLE_ADDR] = place[2]
        df.at[idx, COL_NAME_GOOGLE_CAT0] = place[3][0]
        df.at[idx, COL_NAME_GOOGLE_CAT1] = place[3][1]
        df.at[idx, COL_NAME_GOOGLE_CAT2] = place[3][2]
    return df


Check if it works on a small data-set first


In [189]:
addColumnsForPlaces(df.head(5))


,Merchant,Suburb,GPlace_Name,GPlace_Id,GAddr,GCat0,GCat1,GCat2
0,HARVIN SPICES,HURSTVILLE BC,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
1,DELISSE,SYDNEY,Delisse,ChIJsWvvmWquEmsR_6P02hd1eGQ,"52 Martin Place, Sydney",cafe,bakery,restaurant
2,SOUTHGATE GROUP AUSTRA,SYDNEY,West End Motor Group Pty Ltd,ChIJ8QmHpuaYEmsRMprnJNboLQo,Blacktown,car_dealer,store,UNCATEGORIZED
3,Deli Ziosa,SYDNEY,Deli Ziosa,ChIJE4-sEUCuEmsRSmjjt4PXDlE,"Shop 6.17, MLC Food Court, 19/21 Martin Place,...",meal_takeaway,restaurant,food
5,BING LEE HURSTVILLE,HURSTVILLE BC,Bing Lee Hurstville,ChIJoyx_8aK5EmsR-GUkvCsLgTc,"3/124 Forest Road, Hurstville",electronics_store,home_goods_store,store


### Transform entire dataset
I ran into an strange issue where values were misaligned when running transformation on the entire set. So, I am going to split them into groups and update in small batches

In [204]:
totalSize = len(df)
start = 0
batchSize = 10
transformedBatches = []
while(start<totalSize):
    end = start + batchSize
    batch = df[start:end]
    transformedBatches.append(addColumnsForPlaces(batch))
    start = end
    
transformed = pd.concat(transformedBatches)

## Eyeballing results to make sure they make some sense


In [239]:
transformed[:]

,Merchant,Suburb,GPlace_Name,GPlace_Id,GAddr,GCat0,GCat1,GCat2
0,HARVIN SPICES,HURSTVILLE BC,UNKNOWN,UNKNOWN,UNKNOWN,grocery_or_supermarket,UNCATEGORIZED,UNCATEGORIZED
1,DELISSE,SYDNEY,Delisse,ChIJsWvvmWquEmsR_6P02hd1eGQ,"52 Martin Place, Sydney",cafe,bakery,restaurant
2,SOUTHGATE GROUP AUSTRA,SYDNEY,West End Motor Group Pty Ltd,ChIJ8QmHpuaYEmsRMprnJNboLQo,Blacktown,car_dealer,store,UNCATEGORIZED
3,Deli Ziosa,SYDNEY,Deli Ziosa,ChIJE4-sEUCuEmsRSmjjt4PXDlE,"Shop 6.17, MLC Food Court, 19/21 Martin Place,...",meal_takeaway,restaurant,food
5,BING LEE HURSTVILLE,HURSTVILLE BC,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
6,LA CANTINA ON KENT,SYDNEY,La Cantina Café North Sydney,ChIJU1q2xOyuEmsRUs023sEMv00,"168 Walker Street, North Sydney",cafe,food,UNCATEGORIZED
7,DIRECT DEBIT 142619 TPG INTERNET DH9Q0HB22EP03...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
8,LUO SONG CHICKEN QPS PEAKHURST AUS,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
9,PLINE PH SUTHERLAND,SUTHERLAND,Priceline Pharmacy Interchange,ChIJSe6UidOoEmsRbVGylc3c6vk,"40/436 Victoria Avenue, Chatswood",pharmacy,health,store
10,UBER TRIP WJAQC HELPU 14518236738 AU AUS,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED


#### Manually edit the missing pieces

In [248]:
uncategorizedDf = transformed[transformed[COL_NAME_GOOGLE_CAT0] == 'UNCATEGORIZED']
print uncategorizedDf.shape
transformed.to_csv(DATASET_OUTPUT_FILE_MERCHANT_CATEGORIES)

(88, 8)


In [211]:

uncategorizedDf

,Merchant,Suburb,GPlace_Name,GPlace_Id,GAddr,GCat0,GCat1,GCat2
0,HARVIN SPICES,HURSTVILLE BC,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
5,BING LEE HURSTVILLE,HURSTVILLE BC,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
7,DIRECT DEBIT 142619 TPG INTERNET DH9Q0HB22EP03...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
8,LUO SONG CHICKEN QPS PEAKHURST AUS,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
10,UBER TRIP WJAQC HELPU 14518236738 AU AUS,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
14,PHARMACY DEPOT HURSQPS,HURSTVILLE BC,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
16,TRANSFER TO OTHER BANK NETBANK RENT PRESIDENT AVE,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
17,DIRECT DEBIT 142619 TPG INTERNET DH9M1IFK4EH03...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
18,WDL ATM CBA ATM WYNYARD A NSW 200901 AUS,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
19,LONGO GROUP PTY LTD GYMEA AUS,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNCATEGORIZED,UNCATEGORIZED,UNCATEGORIZED
